In [87]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_transformer
import statistics as sts
import numpy as np
import tensorflow as tf


In [5]:
dataset = pd.read_csv('soybean.csv', sep = ',')
dataset

,date,plant-stand,precip,temp,hail,crop-hist,area-damaged,severity,seed-tmt,germination,...,sclerotia,fruit-pods,fruit-spots,seed,mold-growth,seed-discolor,seed-size,shriveling,roots,class
0,october,normal,gt-norm,norm,yes,same-lst-yr,low-areas,pot-severe,none,90-100,...,absent,norm,dna,norm,absent,absent,norm,absent,norm,diaporthe-stem-canker
1,august,normal,gt-norm,norm,yes,same-lst-two-yrs,scattered,severe,fungicide,80-89,...,absent,norm,dna,norm,absent,absent,norm,absent,norm,diaporthe-stem-canker
2,july,normal,gt-norm,norm,yes,same-lst-yr,scattered,severe,fungicide,lt-80,...,absent,norm,dna,norm,absent,absent,norm,absent,norm,diaporthe-stem-canker
3,july,normal,gt-norm,norm,yes,same-lst-yr,scattered,severe,none,80-89,...,absent,norm,dna,norm,absent,absent,norm,absent,norm,diaporthe-stem-canker
4,october,normal,gt-norm,norm,yes,same-lst-two-yrs,scattered,pot-severe,none,lt-80,...,absent,norm,dna,norm,absent,absent,norm,absent,norm,diaporthe-stem-canker
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
678,april,?,?,?,?,?,upper-areas,?,?,?,...,?,?,?,?,?,?,?,?,?,2-4-d-injury
679,april,lt-normal,?,lt-norm,?,diff-lst-year,scattered,?,?,?,...,?,dna,?,?,?,?,?,?,rotted,herbicide-injury
680,june,lt-normal,?,lt-norm,?,diff-lst-year,scattered,?,?,?,...,?,dna,?,?,?,?,?,?,rotted,herbicide-injury
681,april,lt-normal,?,lt-norm,?,same-lst-yr,whole-field,?,?,?,...,?,dna,?,?,?,?,?,?,rotted,herbicide-injury


In [6]:
dataset.isnull().sum()

date               0
plant-stand        0
precip             0
temp               0
hail               0
crop-hist          0
area-damaged       0
severity           0
seed-tmt           0
germination        0
plant-growth       0
leaves             0
leafspots-halo     0
leafspots-marg     0
leafspot-size      0
leaf-shread        0
leaf-malf          0
leaf-mild          0
stem               0
lodging            0
stem-cankers       0
canker-lesion      0
fruiting-bodies    0
external-decay     0
mycelium           0
int-discolor       0
sclerotia          0
fruit-pods         0
fruit-spots        0
seed               0
mold-growth        0
seed-discolor      0
seed-size          0
shriveling         0
roots              0
class              0
dtype: int64

In [10]:
#descreve elementos de uma coluna
def agrupa(b):
    print(dataset.groupby([b]).size())

In [11]:
agrupa('precip')

precip
?           38
gt-norm    459
lt-norm     74
norm       112
dtype: int64


In [13]:
def moda(i):
  moda = sts.mode(dataset[i])
  dataset[i].fillna(moda, inplace = True)

In [16]:
dataset.loc[dataset['precip'].isin(['?']), 'precip'] = moda('precip')

In [17]:
agrupa('precip')

precip
gt-norm    459
lt-norm     74
norm       112
dtype: int64


In [18]:
agrupa('temp')

temp
?           30
gt-norm    199
lt-norm     80
norm       374
dtype: int64


In [19]:
dataset.loc[dataset['temp'].isin(['?']), 'temp'] = moda('temp')
agrupa('temp')

temp
gt-norm    199
lt-norm     80
norm       374
dtype: int64


In [35]:
dataset.columns

Index(['date', 'plant-stand', 'precip', 'temp', 'hail', 'crop-hist',
       'area-damaged', 'severity', 'seed-tmt', 'germination', 'plant-growth',
       'leaves', 'leafspots-halo', 'leafspots-marg', 'leafspot-size',
       'leaf-shread', 'leaf-malf', 'leaf-mild', 'stem', 'lodging',
       'stem-cankers', 'canker-lesion', 'fruiting-bodies', 'external-decay',
       'mycelium', 'int-discolor', 'sclerotia', 'fruit-pods', 'fruit-spots',
       'seed', 'mold-growth', 'seed-discolor', 'seed-size', 'shriveling',
       'roots', 'class'],
      dtype='object')

In [37]:
for i in dataset.columns:
  for j in dataset[:]:
    dataset.loc[dataset[j].isin(['?']), i] = moda(i)

In [38]:
dataset

,date,plant-stand,precip,temp,hail,crop-hist,area-damaged,severity,seed-tmt,germination,...,sclerotia,fruit-pods,fruit-spots,seed,mold-growth,seed-discolor,seed-size,shriveling,roots,class
0,october,normal,gt-norm,norm,yes,same-lst-yr,low-areas,pot-severe,none,90-100,...,absent,norm,dna,norm,absent,absent,norm,absent,norm,diaporthe-stem-canker
1,august,normal,gt-norm,norm,yes,same-lst-two-yrs,scattered,severe,fungicide,80-89,...,absent,norm,dna,norm,absent,absent,norm,absent,norm,diaporthe-stem-canker
2,july,normal,gt-norm,norm,yes,same-lst-yr,scattered,severe,fungicide,lt-80,...,absent,norm,dna,norm,absent,absent,norm,absent,norm,diaporthe-stem-canker
3,july,normal,gt-norm,norm,yes,same-lst-yr,scattered,severe,none,80-89,...,absent,norm,dna,norm,absent,absent,norm,absent,norm,diaporthe-stem-canker
4,october,normal,gt-norm,norm,yes,same-lst-two-yrs,scattered,pot-severe,none,lt-80,...,absent,norm,dna,norm,absent,absent,norm,absent,norm,diaporthe-stem-canker
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
678,september,normal,gt-norm,norm,yes,same-lst-sev-yrs,whole-field,pot-severe,none,80-89,...,absent,norm,absent,norm,absent,absent,norm,absent,norm,2-4-d-injury
679,september,normal,gt-norm,norm,yes,same-lst-sev-yrs,whole-field,pot-severe,none,80-89,...,absent,norm,absent,norm,absent,absent,norm,absent,rotted,herbicide-injury
680,september,normal,gt-norm,norm,yes,same-lst-sev-yrs,whole-field,pot-severe,none,80-89,...,absent,norm,absent,norm,absent,absent,norm,absent,rotted,herbicide-injury
681,september,normal,gt-norm,norm,yes,same-lst-sev-yrs,whole-field,pot-severe,none,80-89,...,absent,norm,absent,norm,absent,absent,norm,absent,rotted,herbicide-injury


In [70]:
x = dataset.iloc[:, 1:35]
y = dataset.iloc[:, 35]

In [56]:
#label encoder coluna checking_status
#atribui valores de zero a 3
for n in range(35):
  labelencoder = LabelEncoder()
  x[:,n] = labelencoder.fit_transform(x[:,n])
x

array([[0, 1, 0, ..., 'norm', 'absent', 'norm'],
       [0, 1, 0, ..., 'norm', 'absent', 'norm'],
       [0, 1, 0, ..., 'norm', 'absent', 'norm'],
       ...,
       [0, 1, 0, ..., 'norm', 'absent', 'rotted'],
       [0, 1, 0, ..., 'norm', 'absent', 'rotted'],
       [0, 1, 0, ..., 'norm', 'absent', 'rotted']], dtype=object)

In [63]:
#separação entre conjuntos de treino e teste
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)
print(len(x_train), len(x_test), len(y_train), len(y_test))

546 137 546 137


In [84]:
#excluímos a variável para evitar a dummy variable strap
x = x[:]
x

,plant-stand,precip,temp,hail,crop-hist,area-damaged,severity,seed-tmt,germination,plant-growth,...,int-discolor,sclerotia,fruit-pods,fruit-spots,seed,mold-growth,seed-discolor,seed-size,shriveling,roots
0,normal,gt-norm,norm,yes,same-lst-yr,low-areas,pot-severe,none,90-100,abnorm,...,none,absent,norm,dna,norm,absent,absent,norm,absent,norm
1,normal,gt-norm,norm,yes,same-lst-two-yrs,scattered,severe,fungicide,80-89,abnorm,...,none,absent,norm,dna,norm,absent,absent,norm,absent,norm
2,normal,gt-norm,norm,yes,same-lst-yr,scattered,severe,fungicide,lt-80,abnorm,...,none,absent,norm,dna,norm,absent,absent,norm,absent,norm
3,normal,gt-norm,norm,yes,same-lst-yr,scattered,severe,none,80-89,abnorm,...,none,absent,norm,dna,norm,absent,absent,norm,absent,norm
4,normal,gt-norm,norm,yes,same-lst-two-yrs,scattered,pot-severe,none,lt-80,abnorm,...,none,absent,norm,dna,norm,absent,absent,norm,absent,norm
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
678,normal,gt-norm,norm,yes,same-lst-sev-yrs,whole-field,pot-severe,none,80-89,norm,...,none,absent,norm,absent,norm,absent,absent,norm,absent,norm
679,normal,gt-norm,norm,yes,same-lst-sev-yrs,whole-field,pot-severe,none,80-89,norm,...,none,absent,norm,absent,norm,absent,absent,norm,absent,rotted
680,normal,gt-norm,norm,yes,same-lst-sev-yrs,whole-field,pot-severe,none,80-89,norm,...,none,absent,norm,absent,norm,absent,absent,norm,absent,rotted
681,normal,gt-norm,norm,yes,same-lst-sev-yrs,whole-field,pot-severe,none,80-89,norm,...,none,absent,norm,absent,norm,absent,absent,norm,absent,rotted


In [85]:
#labelencoder com a classe
labelencoder_y = LabelEncoder()
y = labelencoder_y.fit_transform(y)
y

array([10, 10, 10, 10, 10, 10, 10, 10, 10, 10,  7,  7,  7,  7,  7,  7,  7,
        7,  7,  7, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 15, 15, 15, 15,
       15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15,
       15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15,
       15, 15,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,
        6,  6,  6,  6,  6, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 11, 11,
       11, 11, 11, 11, 11, 11, 11, 11,  5,  5,  5,  5,  5,  5,  5,  5,  5,
        5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,
        5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
       17, 17, 17, 17, 17, 17, 17, 17, 17, 17,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2, 14, 14, 14, 14,
       14, 14, 14, 14, 14, 14,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1

In [86]:
#Feature Scalling, Padronização z-score
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)
x_test

ValueError: ignored

In [88]:
x_train = tf.convert_to_tensor(x, dtype=tf.int64)

ValueError: ignored

In [75]:
classifier = Sequential()
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 634))
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train, y_train, batch_size = 10, epochs = 1000)

ValueError: ignored